<a href="https://colab.research.google.com/github/GAvila99/Imersao_IA_Alura/blob/main/resumo_diario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install google-genai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
!pip install requests==2.32.3

In [7]:
# =============================================================================
# Projeto: Meu Resumo Local Diário - Configuração Inicial
# =============================================================================

# --- Instalar Bibliotecas Necessárias ---
# Se estiver usando uma versão recente do Colab, 'google-cloud-aiplatform'
# geralmente inclui as bibliotecas ADK e GenAI.
# Caso contrário, você pode precisar instalar individualmente:
# !pip install google-cloud-aiplatform google-colab google-generativeai textwrap
# ou apenas:
!pip install -q google-cloud-aiplatform google-colab

print("Dependências verificadas/instaladas.")

# --- Importações ---
import os
from google.colab import userdata # Usado para carregar segredos no Google Colab
from google.adk.agents import Agent # Importa a classe base para criar agentes
from google.adk.runners import Runner # Usado para executar agentes
from google.adk.sessions import InMemorySessionService # Serviço de sessão em memória para o runner
# Nota: A ferramenta Google Search geralmente é importada diretamente do pacote tools
from google.adk.tools import google_search # Ferramenta para realizar buscas no Google
from google.genai import types # Tipos de dados usados pelo GenAI
from IPython.display import display, Markdown # Para exibir output formatado em Notebooks
import textwrap # Para formatar texto

print("Importações realizadas.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
Dependências verificadas/instaladas.
Importações realizadas.


In [8]:
# =============================================================================
# Configuração da Chave de API
# =============================================================================

# É CRUCIAL manter sua chave de API segura e NUNCA publicá-la diretamente no código.
# No Google Colab, a forma mais segura de fazer isso é usando Segredos (Secrets).
# 1. No menu à esquerda do Colab, clique no ícone de chave (Secrets).
# 2. Clique em '+ New secret'.
# 3. No campo 'Name', digite: GOOGLE_API_KEY
# 4. No campo 'Value', cole a sua chave da API do Google Generative AI.
# 5. Certifique-se de que a opção 'Notebook access' esteja marcada como ON para este notebook.

try:
    # Tenta carregar a chave de API dos segredos do Google Colab
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("Chave de API do Google carregada com sucesso dos segredos do Colab.")
except userdata.NotInColabError:
    # Mensagem caso não esteja rodando no Google Colab
    print("Aviso: Não está no ambiente Google Colab. Certifique-se de configurar a variável de ambiente GOOGLE_API_KEY manualmente.")
    # Se estiver rodando localmente ou em outro ambiente, você precisará definir a variável de ambiente GOOGLE_API_KEY.
    # Exemplo (NÃO recomendado para chaves diretamente no código em produção!):
    # os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_DA_API_AQUI" # <-- COLOQUE SUA CHAVE AQUI SE NÃO ESTIVER USANDO COLAB SECRETS
    # LEMBRE-SE: A forma mais segura é usar variáveis de ambiente ou um gerenciador de segredos.
except Exception as e:
    print(f"Erro ao carregar a chave de API do Google: {e}")
    print("Por favor, verifique se a chave 'GOOGLE_API_KEY' está configurada corretamente nos segredos do Colab.")

# Define o modelo de linguagem a ser usado pelos agentes
# 'gemini-2.0-flash' é um bom modelo para começar, equilibrando custo e desempenho.
MODEL_ID = "gemini-2.0-flash"
print(f"Modelo de linguagem definido: {MODEL_ID}")

Chave de API do Google carregada com sucesso dos segredos do Colab.
Modelo de linguagem definido: gemini-2.0-flash


In [21]:
import asyncio


# =============================================================================
# Funções Auxiliares
# =============================================================================

async def call_agent(agent: Agent, message_text: str) -> str:
    """
    Função para executar um agente ADK e obter sua resposta final.

    Args:
        agent: O objeto Agent a ser executado.
        message_text: A mensagem de entrada para o agente.

    Returns:
        Uma string contendo a resposta final do agente.
    """
    session_service = InMemorySessionService()
    # Note: Usando IDs fixos por simplicidade neste exemplo
    session = await session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera sobre os eventos retornados pelo runner para capturar a resposta final
    try:
      async for event in runner.run(user_id="user1", session_id="session1", new_message=content):
          # Verifica se é a resposta final do agente
          if event.is_final_response():
              # Concatena as partes de texto da resposta
              for part in event.content.parts:
                  if part.text is not None:
                      final_response += part.text
              # final_response += "\n" # Removido quebra de linha extra para síntese
          # Opcional: Imprimir eventos para debug
          # print(f"Evento: {event}")
    except Exception as e:
        print(f"Erro durante a execução do agente {agent.name}: {e}")
        return "Erro na execução do agente."


    # Adiciona uma quebra de linha no final da resposta para melhor separação visual
    if final_response:
        final_response += "\n"

    return final_response.strip() # Remove espaços em branco no início/fim

def to_markdown(text):
    """
    Formata texto para exibição em Markdown, substituindo marcadores e adicionando indentação.

    Args:
        text: O texto a ser formatado.

    Returns:
        Um objeto Markdown para exibição.
    """
    text = text.replace('•', '  *') # Substitui marcadores de lista comuns por markdown
    # Adiciona indentação ao texto para uma apresentação mais limpa
    # return Markdown(textwrap.indent(text, '', predicate=lambda line: True)) # Mantém o texto como está para markdown
    return Markdown(text) # Simplesmente retorna como Markdown para a exibição padrão do Colab

print("Funções auxiliares definidas.")

Funções auxiliares definidas.


In [22]:
# =============================================================================
# Componentes do Projeto (Funções Python)
# =============================================================================

# --- Componente 1: Coleta de Preferências do Usuário (Função Python) ---
def coletar_preferencias_usuario():
    """
    Coleta a localização e o tipo de informação desejada pelo usuário.

    Returns:
        Uma tupla contendo a localização (str) e o tipo de informação (str).
    """
    print("\n" + "=" * 40) # Separador visual
    print("--- Bem-vindo ao Seu Resumo Local Diário! ---")
    print("Vamos coletar algumas informações para gerar seu resumo.")
    print("=" * 40) # Separador visual

    localizacao = input("Por favor, digite sua cidade (ex: Rio de Janeiro): ")
    tipo_informacao = input("Que tipo de informação local você gostaria de receber hoje? (ex: Notícias, Previsão do tempo, Eventos, Promoções): ")

    print("-" * 40) # Linha separadora para clareza
    return localizacao.strip(), tipo_informacao.strip() # Remove espaços em branco

# --- Componente 4: Apresentador (Função Python) ---
def apresentar_resumo(resumo: str):
    """
    Apresenta o resumo final ao usuário em formato Markdown.

    Args:
        resumo: A string contendo o resumo a ser exibido.
    """
    print("\n" + "=" * 40) # Separador visual
    print("--- Seu Resumo Diário Local ---")
    print("=" * 40) # Separador visual

    # Usa a função auxiliar para formatar e exibir em Markdown
    display(to_markdown(resumo))

    print("-" * 40) # Linha separadora final
    print("Fim do Resumo Diário.")
    print("-" * 40) # Linha separadora final

print("Funções de coleta e apresentação definidas.")

Funções de coleta e apresentação definidas.


In [23]:
# =============================================================================
# Componentes do Projeto (Agentes ADK)
# =============================================================================

# --- Componente 2: Agente de Busca Local (Agente ADK com Google Search) ---
# A ferramenta Google Search já foi importada na seção de importações iniciais como 'google_search'

def agente_busca_local(localizacao: str, tipo_informacao: str):
    """
    Agente que utiliza a ferramenta Google Search para buscar informações locais.

    Args:
        localizacao: A cidade ou área para buscar informações.
        tipo_informacao: O tipo de informação a ser buscada (ex: Notícias, Previsão do tempo).

    Returns:
        O resultado da busca como uma string de texto contendo snippets relevantes.
    """


    # Cria a instrução detalhada para o agente de busca
    instrucao_busca = f"""
    Você é um assistente especializado em buscar informações locais recentes usando a ferramenta de busca do google (Google Search).
    Sua tarefa é realizar uma busca precisa por '{tipo_informacao} em {localizacao}'.
    Foque em encontrar resultados que sejam atualizados e relevantes para a localização e o tipo de informação solicitados.
    Retorne os snippets (trechos curtos) dos resultados da busca que contêm as informações mais importantes e diretas sobre o tópico.
    Não tente interpretar, resumir ou analisar a informação neste momento; apenas extraia e apresente os trechos relevantes da busca.
    Se não encontrar resultados relevantes, indique isso claramente.
    """

    # Define o Agente de Busca Local com a ferramenta Google Search
    # CORREÇÃO: Nome do agente alterado para ser um identificador válido
    buscador_local = Agent(
        name="Agente_Busca_Local", # Alterado de "Agente Busca Local"
        model=MODEL_ID, # Usa o modelo de linguagem definido
        instruction=instrucao_busca,
        description="Agente que busca informações locais na web usando Google Search e retorna snippets relevantes.",
        # Use o objeto google_search que foi importado anteriormente, não tente instanciar GoogleSearch
        tools=[google_search]
    )

    # A entrada para este agente é o comando para realizar a busca específica
    entrada_do_agente = f"Realize a busca por '{tipo_informacao} em {localizacao}' e apresente os snippets relevantes."

    # Executa o agente de busca usando a função auxiliar
    resultados_busca = call_agent(buscador_local, entrada_do_agente)

    return resultados_busca

# --- Componente 3: Agente de Análise e Síntese (Agente ADK para processar texto) ---
def agente_analise_sintese(resultados_busca: str, localizacao: str, tipo_informacao: str):
    """
    Agente que analisa os resultados da busca e sintetiza um resumo.

    Args:
        resultados_busca: A string contendo os snippets dos resultados da busca.
        localizacao: A localização usada na busca.
        tipo_informacao: O tipo de informação buscada.

    Returns:
        Uma string contendo o resumo formatado para o usuário.
    """

    # Verifica se os resultados da busca são vazios ou indicam falha
    # Esta verificação pode ser feita antes de chamar o agente para economizar recursos,
    # mas mantive aqui para seguir a lógica original. O agente também é instruído a lidar com isso.
    if not resultados_busca or "não consegui encontrar" in resultados_busca.lower() or "nenhum resultado" in resultados_busca.lower():
          # Retorna uma mensagem informativa se não houver resultados válidos para sintetizar
          return f"Desculpe, não consegui encontrar informações recentes sobre '{tipo_informacao}' em '{localizacao}' no momento com os resultados disponíveis para criar um resumo."


    # Cria a instrução detalhada para o agente de análise e síntese
    instrucao_analise = f"""
    Você recebeu trechos (snippets) de resultados de busca sobre '{tipo_informacao} em {localizacao}'.
    Sua tarefa é ler cuidadosamente esses trechos, identificar as informações mais importantes, relevantes e atualizadas contidas neles, e sintetizá-las em um resumo conciso, claro e fácil de entender para o usuário.
    O resumo deve ser apresentado de forma amigável, como um "Resumo Diário" focado em '{tipo_informacao}' para a área de '{localizacao}'.
    Comece o resumo de forma direta, indicando o tópico e a localização.
    Inclua os pontos principais que você conseguir extrair dos snippets, usando marcadores se apropriado (como '*'). Evite incluir informações irrelevantes ou repetidas.
    Se os trechos forem insuficientes ou confusos, indique que as informações foram limitadas ou não totalmente claras.
    Não invente informações que não estejam presentes nos snippets fornecidos.
    """

    # Define o Agente de Análise e Síntese. Este agente não usa ferramentas externas.
    # CORREÇÃO: Nome do agente alterado para ser um identificador válido
    analisador_resumo = Agent(
        name="Agente_Analise_Sintese", # Alterado de "Agente Análise Síntese"
        model=MODEL_ID, # Usa o modelo de linguagem para análise e síntese
        instruction=instrucao_analise,
        description="Agente que analisa resultados de busca e cria um resumo conciso e relevante.",
        tools=[] # Este agente opera apenas sobre a entrada de texto, não usa ferramentas
    )

    # A entrada para este agente são os resultados brutos (snippets) da busca
    entrada_do_agente = f"Aqui estão os snippets dos resultados da busca sobre '{tipo_informacao} em {localizacao}':\n\n{resultados_busca}\n\nPor favor, analise esses snippets e crie um resumo diário para o usuário."

    # Executa o agente de análise e síntese usando a função auxiliar
    resumo_formatado = call_agent(analisador_resumo, entrada_do_agente)

    return resumo_formatado

print("Funções dos agentes definidas.")

Funções dos agentes definidas.


In [24]:
# =============================================================================
# Lógica Principal de Execução
# =============================================================================

# Este bloco só será executado quando o script for rodado diretamente (ou seja,
# quando esta célula for executada no Colab).

print("\n--- Iniciando Lógica Principal ---")

# 1. Coleta as preferências do usuário (Localização e Tipo de Informação)
localizacao_usuario, tipo_informacao_usuario = coletar_preferencias_usuario()

# Verifica se o usuário forneceu ambas as informações necessárias
if localizacao_usuario and tipo_informacao_usuario:
    # 2. Executa o Agente de Busca Local para obter snippets relevantes
    resultados_da_busca = agente_busca_local(localizacao_usuario, tipo_informacao_usuario)

    # Adiciona uma verificação aqui para não chamar o agente de síntese se a busca falhou
    if resultados_da_busca and "não consegui encontrar" not in resultados_da_busca.lower() and "nenhum resultado" not in resultados_da_busca.lower():
        # 3. Executa o Agente de Análise e Síntese para criar o resumo a partir dos resultados da busca
        resumo_diario = agente_analise_sintese(resultados_da_busca, localizacao_usuario, tipo_informacao_usuario)

        # 4. Apresenta o resumo formatado para o usuário
        apresentar_resumo(resumo_diario)
    else:
         print("\n" + "=" * 40)
         print("--- Não foi possível gerar o Resumo Diário ---")
         print(f"Não foram encontrados resultados relevantes para '{tipo_informacao_usuario}' em '{localizacao_usuario}'.")
         print("=" * 40)


else:
    # Mensagem caso faltem informações essenciais
    print("\nErro: Localização e/ou tipo de informação não foram fornecidos. Não foi possível gerar o resumo.")

print("\n--- Fim da Lógica Principal ---")


--- Iniciando Lógica Principal ---

--- Bem-vindo ao Seu Resumo Local Diário! ---
Vamos coletar algumas informações para gerar seu resumo.
Por favor, digite sua cidade (ex: Rio de Janeiro): Rio de Janeiro
Que tipo de informação local você gostaria de receber hoje? (ex: Notícias, Previsão do tempo, Eventos, Promoções): Noticia
----------------------------------------


AttributeError: 'coroutine' object has no attribute 'lower'

In [13]:
!pip install -U requests

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
